In [1]:
!pip install datasets transformers[sentencepiece]

In [2]:
import pandas as pd
import numpy as np

Bu dataseti ile NLP modelleri kurmustuk. Burda ise ayni datasetinden hazir BERT modelleri ile skor aldik. 

In [20]:
df = pd.read_csv("/content/drive/MyDrive/Womens Clothing E-Commerce Reviews.csv")
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Feature isimlerini degistirdik :

In [7]:
df.rename(columns={"Review Text":"text", "Recommended IND":"label"}, inplace=True)

In [8]:
df=df[['text','label']]
df.head()

,text,label
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


Text verilerinde NaN ifadeler doldurulmaz bu yuzden bunlari düsürdük :

In [9]:
df.dropna(inplace = True)
df

,text,label
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


__[0]["label"] -->__ Cikti iki indexten (label ve score) olustugu icin biz 0. indexteki labelda gecen POSITIVE veya NEGATIVE bilgisini cekmek icin bu sekilde yazdik. Bu islem ornek bir gösterim, asagida bu uygulamayi for döngüsü ile tum dataya uygulayacagiz :

In [14]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")[0]["label"]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


'POSITIVE'

y_pred isimli bosbir liste tanimladik. for dongusu ile her seferinde text icindeki yorumun labelini cek, POSITIVE ise liste icine 1 ekle, degilse 0 ekle dedik. Bu sekilde hazir model uzerinden y_pred' i olusturmus olduk :

In [15]:
y_pred = [1 if classifier(i)[0]["label"] == "POSITIVE" else 0 for i in df["text"]]

#for i in df["text"]:
  #if classifier(i)[0]["label"] == "POSITIVE":
    #y_pred.append(1)
  #else:
    #y_pred.append(0)

Gercek deger ile y_predi karsilastirmak icin eval fonksiyonunu tanimladik :

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
def eval(y, y_pred):
    print("Test_Set")
    print(confusion_matrix(y, y_pred))
    print(classification_report(y, y_pred))
    

Skorlar cok iyi degil. Kullandigimiz bu hazir model, kitaplar uzerinde egitilmis; buradaki datamiz ise kiyafetler uzerine. Modelin bu dataseti ile cok iyi bir tahmin yapamamasi normal. haggingface.co' dan  baska hazir modeller denenerek daha iyi skorlar alinabilir :

In [18]:
print("LOG MODEL")
eval(df["label"], y_pred)

LOG MODEL
Test_Set
[[ 3209   892]
 [ 2756 15784]]
              precision    recall  f1-score   support

           0       0.54      0.78      0.64      4101
           1       0.95      0.85      0.90     18540

    accuracy                           0.84     22641
   macro avg       0.74      0.82      0.77     22641
weighted avg       0.87      0.84      0.85     22641

